### Pacotes Usados

In [1]:
import os

In [2]:
import sys

In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
from time import time 

In [6]:
sys.path.append(os.path.abspath('../src/'))

In [7]:
from knap_sack import by_minimu_weights, by_greedy_benefit_cust,by_benefit_cust_with_lru, by_benefit_cust_with_table

### Atualizando o limite de recursividade

In [8]:
sys.setrecursionlimit(100000000)

## Carregamento dos Dados

### Funções Auxiliares de Carregmento dados 

In [9]:
def load_data_itens_weights(name:str) -> tuple:
    data = pd.DataFrame(
            pd.read_csv(
            '../dados_processados/' + name,
            sep=' '
        )
      )
    data = data.reset_index()
    data = data.rename(columns={data.columns[0]: 'indice', data.columns[1]: "value", data.columns[2]: 'weight'})
    data = data.sort_values(by='weight')
    return data

In [10]:
def load_data_itens_benefit_by_cust(name:str) -> pd.DataFrame:
    dataframe:pd.DataFrame = pd.DataFrame(
        pd.read_csv(
          '../dados_processados/'+name,
          sep=' ',
          header=None
      )
    )
    
    dataframe = dataframe.rename(columns={dataframe.columns[0]: "value", dataframe.columns[1]: 'weight'})
        
    values:np.array = dataframe.value
    weights:np.array = dataframe.weight
    benefit_by_cust_array:np.array = np.divide(values, weights)
    benefit_by_cust_array = np.trunc(benefit_by_cust_array)

    dataframe['benefit_by_cust'] = benefit_by_cust_array
    dataframe = dataframe.sort_values(by='benefit_by_cust')

    dataframe = dataframe.reset_index()
    dataframe = dataframe.rename(
      columns={
          dataframe.columns[0]: 'indice', 
          dataframe.columns[1]: "value", 
          dataframe.columns[2]: 'weight'
          }
      )
    
    return dataframe

In [11]:
def load_data_benefit(name:str) -> int:
    data:int = 0
    with open('../dados_processados/' + name, mode='r') as file:
        data = int(file.readline())
    return data

In [12]:
def load_data_weight(name:str) -> int:
    data = pd.DataFrame(
        pd.read_csv(
        '../dados_processados/' + name,
        sep=' ',
        )
    )

    weight:int = int(data.columns[1])
    return weight

In [13]:
def load_data_answer(name:str) -> np.array:
    data:np.array = np.array([])
    data = np.loadtxt('../dados_processados/' + name, delimiter=' ')
    return data

In [14]:
def load_data_benefit_by_cust(names:pd.DataFrame, lista_data:list) -> list:
    for _, i in names.iterrows():
        lista_data.append(
            [
                load_data_itens_benefit_by_cust(i.input_file),
                load_data_weight(i.input_file),
                load_data_benefit(i.output_file),
                load_data_answer(i.answer_file)
            ]
        )
    return lista_data

In [15]:
def load_data_minimu_weights(names:pd.DataFrame, lista_data:list) -> list:
    for _, i in names.iterrows():
        lista_data.append(
            [
                load_data_itens_weights(i.input_file),
                load_data_weight(i.input_file),
                load_data_benefit(i.output_file),
                load_data_answer(i.answer_file)
            ]
        )
    return lista_data

In [16]:
name_file:pd.DataFrame = pd.DataFrame(
    pd.read_csv(
        '../dados_pre_processados/name_file.csv',
        sep=',',
    )
)

In [17]:
def save_data_execute(name:str, lista_dados:list) -> None:
    output:np.array = np.array(lista_dados[1])
    time:np.array = np.array(lista_dados[0])
    output.tofile('output' + name + '.csv', sep=',', format='%d')
    time.tofile('output_time' + name + '.csv', sep=',', format='%s')

In [18]:
file_by_dinamic:pd.DataFrame = name_file[name_file.input_file.str.contains('_\d{3}_')]

### Itens usados para o Carregamento dos Dados

In [19]:
lista_data_file_by_weights:list = []

In [20]:
lista_data_file_benefit_by_cust_1:list = []

In [21]:
lista_data_file_benefit_by_cust_2:list = []

In [22]:
lista_data_file_benefit_by_cust_1 = load_data_benefit_by_cust(name_file, lista_data_file_benefit_by_cust_1)

In [23]:
lista_data_file_benefit_by_cust_2 = load_data_benefit_by_cust(file_by_dinamic, lista_data_file_benefit_by_cust_2)

In [24]:
load_data_file_by_weights = load_data_minimu_weights(name_file, lista_data_file_by_weights)

## Execuçaõ do Algoritmos

### Funções Auxiliares de Execução do Algoritmos

#### Função de Execução do Algoritmo Guloso

In [25]:
def execute_by_epoch_g_m(data_op:list) -> tuple:
    i:int = 1
    start_time:float = 0.0
    predict_algorithm:list = []
    p_answer:int  = 0
    time_execute_by_greedy_algorithm:list = []
    
    for data in data_op:
        print(i/20)
        start_time = time()
        p_answer = by_minimu_weights(data[1], data[0])
        time_execute_by_greedy_algorithm.append(
                time() - start_time
        )
        predict_algorithm.append(
                p_answer
        )
        i += 1

    return time_execute_by_greedy_algorithm, predict_algorithm

In [26]:
def execute_by_epoch_g_bc(data_op:list) -> tuple:
    i:int = 1
    p_answer:int = 0
    start_time:float = 0.0
    time_execute:list = []
    predict_algorithm:list = []
  
    for data in data_op:
        print(i/20)
        start_time = time()
        p_answer = by_greedy_benefit_cust(data[1],data[2], data[0])
        time_execute.append(
            time() - start_time
        )
        predict_algorithm.append(
            p_answer
        )
        i += 1

    return time_execute, predict_algorithm

In [27]:
def execute_by_epoch_d(data_op:list) -> tuple:
    i:int = 0
    p_answer:int = 0
    start_time:float = 0.0
    time_execute:list = []
    predict_algorithm:list = []
  
    for data in data_op:
        print(i/len(data_op))
        start_time = time()
        p_answer = by_benefit_cust_with_table(
                data[1],
                data[0].shape[0],
                data[0]
        )
        time_execute.append(
                time() - start_time
        )
        predict_algorithm.append(
                p_answer
        )
        i += 1
      
    return time_execute, predict_algorithm

In [28]:
def execute_by_epoch_dlru(epoch:int, data_op:list) -> tuple:
    p_answer:int = 0
    start_time:float = 0.0
    time_execute:list = []
    predict_algorithm:list = []
    
    for _ in range(epoch):
        for data in data_op:
            start_time = time()
            p_answer = by_benefit_cust_with_lru(
                data[0].shape[0], 
                data[1], 
                tuple(data[0].value.to_list()),
                tuple(data[0].weight.to_list())
            )
            time_execute.append(
                time() - start_time
            )
            predict_algorithm.append(
                p_answer
            )
    return time_execute, predict_algorithm

#### Execuçãos dos algoritmos

In [29]:
_greedy_by_minimum_weight:tuple = execute_by_epoch_g_m(lista_data_file_by_weights)

0.05


/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-P

/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-P

/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-P

/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-P

...

/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-P

/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-P

/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-P

1.05


/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-P

In [30]:
_greedy_by_benefit_cust:tuple = execute_by_epoch_g_bc(lista_data_file_benefit_by_cust_1)

0.05


/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({


0.1
0.15
0.2


/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({


0.25
0.3


...

/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({


0.95
1.0


/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({
/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({


1.05


/home/raullima/Documentos/Git/KnapSack-Problem/src/knap_sack.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  set_solution = set_solution.append({


In [31]:
_dinamic_table:tuple = execute_by_epoch_d(lista_data_file_benefit_by_cust_2)

0.0
0.1111111111111111
0.2222222222222222
0.3333333333333333
0.4444444444444444
0.5555555555555556
0.6666666666666666
0.7777777777777778
0.8888888888888888


In [32]:
_dinamic_lru:tuple = execute_by_epoch_dlru(1, lista_data_file_benefit_by_cust_2)

## Salvar dados

In [33]:
save_data_execute('_dinamic_lru', _dinamic_lru)

In [34]:
save_data_execute('_dinamic_table', _dinamic_table)

In [35]:
save_data_execute('_greedy_by_benefit_cust', _greedy_by_benefit_cust)

In [36]:
save_data_execute('_greedy_by_minimum_weight', _greedy_by_minimum_weight)